# Exercises

**1. Read the source code for `t()` and `t.test()` and confirm that `t.test()` is an S3 generic and not an S3 method. What happens if you create an object with class test and call `t()` with it?**

In [8]:
library(pryr)

print(ftype(t.test))

x <- 1:3
class(x) <- 'test'

print(t(x))

# Running `t(x)` matches to the `t.test(x)` function because I have class `test`

[1] "s3"      "generic"

	One Sample t-test

data:  x
t = 3.4641, df = 2, p-value = 0.07418
alternative hypothesis: true mean is not equal to 0
95 percent confidence interval:
 -0.4841377  4.4841377
sample estimates:
mean of x 
        2 



**2. What classes have a method for the `Math` group generic in base R? Read the source code. How do the methods work?**

**3. R has two classes for representing date time data, `POSIXct` and `POSIXlt`, which both inherit from `POSIXt`. Which generics have different behaviours for the two classes? Which generics share the same behaviour?**

`POSIXct` is a count of time clock since 1970, whereas `POSIXlt` is a list of information about the date (day, month, second, etc). Because of this, some generics cannot have the same behavior. For example, `print(...)` works differently, since `POSIXct` does not have timezone information. Also comparison operators (`<`, `-`, etc.) for `POSIXlt` objects should be handled carefully, since the objects may differ in the fields present.

**4. Which base generic has the greatest number of defined methods?**

In [38]:
library(dplyr)

objs <- mget(ls('package:base'), inherits=TRUE)
funs <- Filter(is.function, objs)
generics <- funs %>%
                sapply(ftype) %>%
                sapply(function(x) x[2])
generics <- Filter(function(x) x == 'generic', generics)
gen.methods <- sapply(names(generics), methods) %>%
               sapply(length) %>%
               sort(decreasing=TRUE)

head(gen.methods)

[ as.character           [[            $            c          [<- 
          35           14           12           12           10            9

**5. `UseMethod()` calls methods in a special way. Predict what the following code will return, then run it and read the help for `UseMethod()` to figure out what’s going on. Write down the rules in the simplest form possible.**

In [41]:
# I predict `g(10)` to return 2.
# I predict `h("a")` to return "char a"

y <- 1
g <- function(x) {
    y <- 2
    UseMethod("g")
}
g.numeric <- function(x) y
g(10)

h <- function(x) {
    x <- 10
    UseMethod("h")
}
h.character <- function(x) paste("char", x)
h.numeric <- function(x) paste("num", x)

h("a")
    
# What's happening:
#    for `g(10)`, since 10 is numeric, it calls `g.numeric`, which returns `y`.
#    `y` is defined as 2, so it returns 2.
#
#    for `h("a")`, it calls `h.character`, which returns `paste("char", x)`. Since
#    `x` is the argument for `h.character`, it uses this as `x`, which is "a", so
#    it returns "char a".

[1] 2

[1] "char a"

**6. Internal generics don’t dispatch on the implicit class of base types. Carefully read `?"internal generic"` to determine why the length of `f` and `g` is different in the example below. What function helps distinguish between the behaviour of `f` and `g`?**

    f <- function() 1
    g <- function() 2
    class(g) <- "function"

    class(f)
    class(g)

    length.function <- function(x) "function"
    length(f)
    length(g)

Internal generics only dispatch on objects, where `is.object()` returns `TRUE`. Looking at the documentation of `is.object()`, this checks for an `OBJECT` bit flag, which is set when the `class<-` is set. When you first define `f` and `g` like above, `is.object(f)` returns `FALSE`, so dispath doesn't work and `length.function()` is not called for `f`. But when you explicitly set the class for `g`, that changes things for `g`, and `length.function()` is called rather than the generic `length`.

**7. Which S4 generic has the most methods defined for it? Which S4 class has the most methods associated with it?**

**8. What happens if you define a new S4 class that doesn’t “contain” an existing class? (Hint: read about virtual classes in `?setClass`.)**

Doing so creates a virtual class, which is a class that can have no instantiations.

**9. What happens if you pass an S4 object to an S3 generic? What happens if you pass an S3 object to an S4 generic? (Hint: read `?setOldClass` for the second case.)**

If you pass an S4 object to an S3 generic, it can use `extends(class(x))` to get the inheritance hierarchy and find the correct method to use. If you pass an S3 object to an S4 generic, it calls `setOldClass` in the constructor of an S4 object, which creates an S4 object with the S3 object as a template.

**10. Use a field function to prevent the account balance from being directly manipulated. (Hint: create a “hidden” `.balance` field, and read the help for the fields argument in `setRefClass()`.)**

TODO

**11. I claimed that there aren’t any RC classes in base R, but that was a bit of a simplification. Use `getClasses()` and find which classes `extend()` from `envRefClass`. What are the classes used for? (Hint: recall how to look up the documentation for a class.)**

In [73]:
library(methods)

classes <- getClasses(where=as.environment('package:base'))
ref.class.mask <- sapply(sapply(classes, extends), function(x) 'envRefClass' %in% x)
print(head(ref.class.mask))

named list()
